## ACTIVATE AUTOMATION CODE

### MUST HAVE 
+ have the Level 3 Mapping in the Contribution Tab, Spends Tab, Support Tab
+ Tabs named **Factor_Code** & **Time_Mapping** present for the Price Margin Creation
+ **If using for Another Project, Need to make changes in code regarding the Time Mapping**

In [70]:
import pandas as pd
import os
from datetime import datetime
import time 

### ACTIVATE INPUT SETUP 
+ Please enter your folder directory where you are having the Input File
+ Specify the Input/Output file name

In [71]:
# SPECIFY HERE
# Path should be having a r before the path 
# Ex : r"C:/Folder1/Activate_Files"
start_time = time.time()
folder = r"C:\Users\Beeraboina.Rahul\OneDrive - Ipsos\Work\Python Code Automation"

# SPECIFY THE NAMES 
input_file = "MCUSC_2025_Annual_Cholula_WB_12.1.25_US.xlsx"
output_file = f"Activate_DataSet_{datetime.now().strftime('%m-%d-%Y')}.xlsx"

Input_path = os.path.join(folder,input_file)
Output_path = os.path.join(folder,output_file)

# CHECKING IF AN OUTPUT FILES EXIST OR NOT
if os.path.exists(Output_path):
    Output = pd.read_excel(Output_path)
    print("✔ Output file found and loaded.")
else:
    # Create an empty file or add specific columns if needed
    Output = pd.DataFrame()
    # Create new Excel file
    Output.to_excel(Output_path, index=False)
    print("⚠ Output file not found. A new one has been created:", Output_path)


✔ Output file found and loaded.


In [72]:
# Read all sheets
sheets = pd.read_excel(Input_path, sheet_name=None)

with pd.ExcelWriter(Output_path, engine="openpyxl") as writer:
    for sheet_name, df in sheets.items():

        # -----------------------------------------
        # TAB 1: Weekly logic
        # -----------------------------------------
        if sheet_name == "Weekly":

            date_columns = df.columns[9:]
            result = df.groupby(["ModelKey", "Level 3"], as_index=False)[date_columns].sum()
            result.columns = result.columns.map(lambda x: x.strftime("%m-%d-%Y") if hasattr(x, "strftime") else x)
            result.to_excel(writer, sheet_name="Contribution", index=False)

        elif sheet_name == "WeeklySupport":

            # Example: group by Category instead
            date_columns = df.columns[3:] 
            result = df.groupby(["Level 3"], as_index=False).sum()
            result.columns = result.columns.map(lambda x: x.strftime("%m-%d-%Y") if hasattr(x, "strftime") else x)
            result.to_excel(writer, sheet_name="Data", index=False)

        # -----------------------------------------
        # TAB 3: Daily logic
        # -----------------------------------------
        elif sheet_name == "WeeklySpend":

            # Example: filter + group
            date_columns = df.columns[2:]
            result = df.groupby("Level 3", as_index=False).sum()
            result.columns = result.columns.map(lambda x: x.strftime("%m-%d-%Y") if hasattr(x, "strftime") else x)
            result.to_excel(writer, sheet_name="WeeklySpend", index=False)

        elif sheet_name == "Activate Mappings":

            CONCAT_MAPPING = sheets["Activate Mappings"].loc[:, ["Type", "Variable", "Variable(Level3)", "Decay", "Learn"]]
            Incremental = CONCAT_MAPPING.loc[CONCAT_MAPPING["Type"] == "Incremental"].copy()
            Incremental.loc[:, "Transformation Formula"] = ("APL([" + Incremental["Variable"].astype(str)+ "],0,"+ Incremental["Decay"].astype(str)+ ","+ Incremental["Learn"].astype(str)+ ",0)")
            Incremental_Level3_Unique = Incremental.drop_duplicates(subset=["Variable(Level3)"])
            Incremental_Level3_Unique.to_excel(writer,sheet_name="Variable_Sheet",index=False)

        elif sheet_name == "Factors":

            # PROJECTION FACTOR SHEET CREATION
            metric_df = sheets["Factor_Code"]
    
            Model_List = sheets["Factor_Code"].ModelKey.unique().tolist()
            KPI = ["ProjUnits","Profit","Revenue"]


            # In the below code, if you are using this code for antoher project, Make changes in the formatting string below.
            # It appends the Cholula for each and every model of the project. It is basically a Cartesian Product of the KPIs and the Model.
            PMCombo= pd.DataFrame([(f"Cholula {m.replace('_', ' ')}", k) for m in Model_List for k in KPI],columns=["Model", "KPI"])
            
            Time_Mapping = sheets["Time Mapping"]
            Time_Mapping["Date"] = pd.to_datetime(Time_Mapping["Date"]).dt.strftime("%m/%d/%Y")

            Date_As_Cols = pd.DataFrame(columns=Time_Mapping.iloc[:,1])
            ModelPriceMargin = pd.concat([PMCombo,Date_As_Cols],axis=1)

            # MAKE CHANGES HERE
            # When using for a new project, need to make changes to the Time Mapping in it as CY52, LY52 & TYA52 are
            # specific for Cholula Project.
            Projection_df = metric_df.loc[metric_df["Metric"] == "PROJECTION",["ModelKey","CY 52","LY52","TYA52"]]

            Projection_long = Projection_df.melt(id_vars="ModelKey",var_name="Mapping",value_name="Value")

            Projection_long = Projection_long.merge(Time_Mapping,on="Mapping",how="left")
            date_order = Time_Mapping["Date"].drop_duplicates().tolist()
            
            Projection_pivot = Projection_long.pivot(index="ModelKey",columns="Date",values="Value").reset_index()
            
            Projection_pivot = Projection_pivot[["ModelKey"] + date_order]
            
            Projection_pivot["Model"] = "Cholula " + Projection_pivot["ModelKey"].str.replace("_"," ") 

            Final_ModelPriceMargin = ModelPriceMargin.merge(Projection_pivot,on="Model",how="left")
            Final_ModelPriceMargin = Final_ModelPriceMargin.loc[:,~Final_ModelPriceMargin.columns.str.endswith("_x")]
            Final_ModelPriceMargin.columns = Final_ModelPriceMargin.columns.str.replace('_y$',"",regex=True)
            Final_ModelPriceMargin = Final_ModelPriceMargin.drop(columns=["ModelKey"])
            Final_ModelPriceMargin.to_excel(writer,sheet_name="ProjectionFactor",index=False)

            # MODEL_PRICE_MARGIN CREATION
            DF_model = pd.DataFrame({"Model":Model_List})
            Time_Mapping["key"] = 1
            DF_model["key"] = 1
            MPM = DF_model.merge(Time_Mapping,on="key").drop(columns="key")
            MPM["Model"] = "Cholula " + MPM["Model"].str.replace("_"," ")
            MPM = MPM.rename(columns={"Model":"Product","Mapping":"TimeDefinition","Date":"Period"})
            MPM["Model Name"] = MPM["Product"] 
            MPM["Geography"] = "All"
            MPM["Channel"] = "All"

            Proper_Order = ["Model Name","Product","Geography","Channel","TimeDefinition","Period"]
            MPM = MPM[Proper_Order]
            pm_df = metric_df.loc[


            # MAKE CHANGES HERE    
            metric_df["Metric"].isin(["PRICE", "MARGIN"]),["Metric", "ModelKey", "CY 52", "LY52", "TYA52"]].copy()
            pm_long = pm_df.melt(id_vars=["Metric", "ModelKey"],var_name="Mapping",value_name="Value")
            pm_long = pm_long.merge(Time_Mapping,on="Mapping",how="left")
            pm_long["Model Name"] = ("Cholula " + pm_long["ModelKey"].str.replace("_", " ", regex=False))
            
            pm_pivot = (pm_long.pivot(index=["Model Name", "Date"],columns="Metric",values="Value").reset_index())
            MPM = MPM.merge(pm_pivot,left_on=["Model Name", "Period"],right_on=["Model Name", "Date"],how="left")
            MPM["Price"] = MPM["PRICE"]
            MPM["Margin"] = MPM["MARGIN"]

            MPM.drop(columns=["PRICE", "MARGIN", "Date"], inplace=True)
            MPM.to_excel(writer,sheet_name="ModelPriceMargin",index=False)

In [73]:
print("ALL DONE") 
end_time = time.time()
execution_time = end_time - start_time 
print(f"Total Execution time: {execution_time:.2f} seconds")

ALL DONE
Total Execution time: 28.93 seconds
